In [1]:
import os

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
model = ChatGoogleGenerativeAI(model = 'gemini-pro',convert_system_message_to_human=True)

In [7]:
model.invoke("Hi").content

e:\ChatbotUsingLangchainWithMemory\myenv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Hello there! How can I assist you today?'

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [10]:
parser.invoke(model.invoke("Hi"))

e:\ChatbotUsingLangchainWithMemory\myenv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Hello there! How can I assist you today?'

In [11]:
from langchain_core.messages import HumanMessage

In [13]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# This is like QA it does not memorize the message.(Eg:I have given my name and in next query i asked my name it does not provide my name)
while True:
    message = input("Write tour query:")
    if message == "Bye":
        print("Good Bye have a great day")
        break
    else:
        print(parser.invoke(model.invoke([HumanMessage(content=message)])))

I am a digital assistant, and do not have personal feelings or the capacity to experience emotions like humans do. However, I am designed to be helpful and informative, and I am here to assist you with any questions or tasks you may have. How can I help you today?
Nice to meet you, Shoaib.
I do not have access to your personal information, so I cannot tell you your name.
Good Bye have a great day


In [15]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

In [16]:
from langchain_core.runnables import RunnableWithMessageHistory

In [17]:
from langchain_core.messages import AIMessage

In [ ]:
# this is called instruction fine tuning, we are giving instructions and then asking the question
result = model.invoke([
    HumanMessage(content="Hi, I'm Shoaib"),
    AIMessage(content="Hello Shoaib! How can I assist you today?"),
    HumanMessage(content="What is my name?")
])

In [19]:
parser.invoke(result)

'Your name is Shoaib.'

## Chatbot with memory

In [20]:
# for storing the session id and chats
store = {}

In [21]:
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [22]:
config = {"configurable":{"session_id":"firstchat"}}

In [23]:
model_with_memory = RunnableWithMessageHistory(model,get_session_history)

In [24]:
model_with_memory.invoke([HumanMessage("Hi, I am Shoaib!")],config=config).content

"Hello, Shoaib! It's nice to meet you. I'm Gemini, a multimodal AI model, developed by Google."

In [26]:
model_with_memory.invoke([HumanMessage("What is my name?")],config=config).content

'Your name is Shoaib, as you mentioned in your previous message.'

In [27]:
config = {"configurable":{"session_id":"secondchat"}}

In [28]:
model_with_memory.invoke([HumanMessage("What is my name?")],config=config).content

'I do not have access to your personal information, including your name.'

In [29]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am Shoaib!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, Shoaib! It's nice to meet you. I'm Gemini, a multimodal AI model, developed by Google.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-59999300-7af8-463c-ba23-754333e9c23e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 27, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs=

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [31]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [32]:
chain = prompt | model 

In [34]:
chain.invoke({"messages":[HumanMessage(content="Hi!, I'am Shoaib")]}).content

"Hi Shoaib, I'm a helpful assistant. I'll do my best to answer all your questions to the best of my ability. Just let me know what you need help with."

In [35]:
chain.invoke({"messages":[HumanMessage(content="What is my name?")]}).content

'I do not have access to your personal information and cannot answer questions about your identity.'

In [36]:
model_with_memory = RunnableWithMessageHistory(chain,get_session_history)

In [37]:
config = {"configurable":{"session_id":"thirdchat"}}

In [39]:
response = model_with_memory.invoke(
    [HumanMessage(content="Hi!,I am Shoaib")],
    config=config
)

In [40]:
response.content

"Hi Shoaib, it's nice to meet you. I am a helpful assistant. How can I help you today?"

In [41]:
response = model_with_memory.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

In [42]:
response.content

'Your name is Shoaib, as you told me earlier. Is there anything else I can help you with today?'

In [43]:
response = model_with_memory.invoke(
    [HumanMessage(content="What is 3+3?")],
    config=config
)

In [44]:
response.content

'3 + 3 is 6.'

In [45]:
response = model_with_memory.invoke(
    [HumanMessage(content="Who is Indain cricket team captain?")],
    config=config
)

In [46]:
response.content

'The current captain of the Indian cricket team is Rohit Sharma.'

In [47]:
response = model_with_memory.invoke(
    [HumanMessage(content="What should I add in previous addition so that it become 10")],
    config=config
)

In [48]:
response.content

'You need to add 4 to the previous addition (6) to make it 10.'

In [49]:
response = model_with_memory.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

In [50]:
response.content

'You told me earlier that your name is Shoaib.'

In [51]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, I am Shoaib!', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello, Shoaib! It's nice to meet you. I'm Gemini, a multimodal AI model, developed by Google.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-59999300-7af8-463c-ba23-754333e9c23e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 27, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='What is my name?', additional_kwargs=

In [52]:
# If we see in store dict it contain lot of messages (context window) trim the chat history.(GPT:128k tokens)
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 40,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

In [53]:
messages = [
    HumanMessage(content="Hi!,I am shoaib"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes")
]

In [54]:
model.get_num_tokens_from_messages(messages)

51

In [56]:
trimmed_message = trimmer.invoke(messages)

In [57]:
model.get_num_tokens_from_messages(trimmed_message)

39

In [ ]:
# here in the output it did not provide me the name bz the message got trimmed.
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What is my name?")],
        "language":"English"
    }
)

response.content

'I do not have access to that information.'

In [59]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What math problem did i ask??")],
        "language":"English"
    }
)

response.content

'2+2'

In [62]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable":{"session_id":"fourthchat"}}

In [64]:
response = model_with_memory.invoke(
    {
        "messages":messages + [HumanMessage(content="What is my name?")],
        "language":"English"
    },
    config=config
)

response.content

'I do not have access to that information.'